In [ ]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import time

In [ ]:
# 📂 1. Load ETTh2 Data from local file
file_path = "../data/raw/ETTh2.csv"

df_etth2 = pd.read_csv(file_path)

print("✅ ETTh2 dataset loaded from local file!")
print(df_etth2.head())

In [ ]:
# 📜 2. Preprocessing
# ❗ IMPORTANT: These min and max values must be same as ETTh1 preprocessing min-max
# Update them according to your preprocessing!

file_path_etth1 = "../data/raw/ETTh1.csv"

df_etth1 = pd.read_csv(file_path_etth1)

# Get min and max for 'OT' column
min_OT = df_etth1['OT'].min()
max_OT = df_etth1['OT'].max()

df_etth2['OT'] = (df_etth2['OT'] - min_OT) / (max_OT - min_OT)

print("\n✅ Preprocessing applied (normalization)!")
print(df_etth2[['date', 'OT']].head())

In [ ]:
# Prepare a list to store results
results = []

# 🚀 FastAPI Server URL
fastapi_url = "http://localhost:8000/predict"

# 🧠 Rolling window for 96 sensor readings
rolling_window = []

# 📈 Stream row by row
for idx, row in df_etth2.iterrows():
    # Add new value to the window
    ot_value = row['OT']
    rolling_window.append(ot_value)

    row = df_etth2.iloc[idx]
    next_row = df_etth2.iloc[idx + 1] if idx + 1 < len(df_etth2) else None

    # Keep only the last 96 points
    if len(rolling_window) > 96:
        rolling_window.pop(0)

    # Predict only when we have 96 points
    if len(rolling_window) == 96:
        # Convert rolling_window values to Python native types (e.g., float)
        payload = {"sensor_values": rolling_window}

        try:
            response = requests.post(fastapi_url, json=payload)
            response.raise_for_status()
            preds = response.json()

            real_ot = next_row['OT'] if next_row is not None else None
            pred_ot = preds['Final_Ensembled_Prediction']

            # Anomaly detection
            error = abs(pred_ot - real_ot) if real_ot is not None else None
            anomaly_detected = error > 0.1 if error is not None else None

            # Predictive maintenance
            predictive_maintenance = pred_ot > 0.8

            # Energy efficiency
            energy_efficiency = pred_ot < 0.6

            # Append results to the list
            result_row = {
                "Time": row['date'],
                "Real OT": real_ot,
                "Predicted OT": pred_ot,
                "Anomaly Detected": anomaly_detected,
                "Predictive Maintenance": predictive_maintenance,
                "Energy Efficiency": energy_efficiency,
                "LSTM Prediction": preds['LSTM_Prediction'],
                "PatchTST Prediction": preds['PatchTST_Prediction'],
                "LLM Prediction": preds['LLM_Prediction']
            }
            results.append(result_row)

            # Convert the result row to a DataFrame
            result_df = pd.DataFrame([result_row])

            # Append the result row to the Excel file
            try:
                with pd.ExcelWriter("../data/results/prediction_results.xlsx", mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
                    result_df.to_excel(writer, index=False, header=writer.sheets.get('Sheet1') is None, startrow=writer.sheets.get('Sheet1').max_row if writer.sheets.get('Sheet1') else 0)
            except FileNotFoundError:
                # If the file does not exist, create it
                with pd.ExcelWriter("../data/results/prediction_results.xlsx", mode='w', engine='openpyxl') as writer:
                    result_df.to_excel(writer, index=False, header=True)

        except requests.exceptions.RequestException as e:
            print(f"❌ Request failed: {e}")

    # ⏳ Wait for 1 second to simulate real-time
    time.sleep(1)

print(f"\n✅ Results saved to {'../data/results/prediction_results.xlsx'} successfully!")